In [1]:
# -*- coding: utf-8 -*-

'''
@Author   :   Corley Tang
@contact  :   cutercorleytd@gmail.com
@Github   :   https://github.com/corleytd
@Time     :   2023-12-03 12:49
@Project  :   Hands-on Crawler with Python-mongo_with_python
Python操作MongoDB
'''

# 导入所需的库
import pymongo
from bson.objectid import ObjectId

# 1.连接MongoDB

In [2]:
client = pymongo.MongoClient(host='127.0.0.1', port=27017)  # 连接MongoDB，等价于pymongo.MongoClient('mongodb://127.0.0.1:27017')
client

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)

# 2.指定数据库和集合

In [3]:
db = client.question_answer  # 指定连接question_answer数据库，等价于client['question_answer']
db

Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'question_answer')

In [4]:
collection = db.user  # 指定连接user集合（表），等价于db['user']
collection

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'question_answer'), 'user')

# 3.插入数据

In [5]:
person = {'name': 'Amanda', 'age': 18, 'gender': 'female'}
result = collection.insert_one(person)
result  # 返回插入数据的结果，包含插入数据的_id

InsertOneResult(ObjectId('656c2e02ab8e650a92de9c24'), acknowledged=True)

In [6]:
result.inserted_id  # 查看插入数据的_id

ObjectId('656c2e02ab8e650a92de9c24')

In [7]:
person1 = {'name': 'Alice', 'age': 24, 'gender': 'female'}
person2 = {'name': 'Bob', 'age': 25, 'gender': 'male'}
person3 = {'name': 'Charlie', 'age': 26, 'gender': 'male'}
result = collection.insert_many([person1, person2, person3])
result

InsertManyResult([ObjectId('656c2e02ab8e650a92de9c25'), ObjectId('656c2e02ab8e650a92de9c26'), ObjectId('656c2e02ab8e650a92de9c27')], acknowledged=True)

In [8]:
result.inserted_ids  # 查看插入数据的_id

[ObjectId('656c2e02ab8e650a92de9c25'),
 ObjectId('656c2e02ab8e650a92de9c26'),
 ObjectId('656c2e02ab8e650a92de9c27')]

# 4.查询数据

In [9]:
# 通过一般字段查询
collection.find_one({'name': 'Amanda'})

{'_id': ObjectId('656c2e02ab8e650a92de9c24'),
 'name': 'Amanda',
 'age': 18,
 'gender': 'female'}

In [10]:
# 通过_id查询
collection.find_one({'_id': ObjectId('656c10d33d74577e649a8660')})

In [11]:
# 查询多条数据
result = collection.find({'age': {'$gt': 22}})
result  # Cursor对象，相当于一个生成器，通过遍历获取每个结果

In [12]:
for item in result:
    print(item)  # 每个结果都是字典

{'_id': ObjectId('656c2e02ab8e650a92de9c25'), 'name': 'Alice', 'age': 24, 'gender': 'female'}
{'_id': ObjectId('656c2e02ab8e650a92de9c26'), 'name': 'Bob', 'age': 25, 'gender': 'male'}
{'_id': ObjectId('656c2e02ab8e650a92de9c27'), 'name': 'Charlie', 'age': 26, 'gender': 'male'}


# 5.计数排序

In [13]:
collection.count_documents({})  # 计数，代替collection.find().count()

4

In [14]:
for item in collection.find().sort('age', pymongo.DESCENDING):
    print(item)  # 按照age降序排序

{'_id': ObjectId('656c2e02ab8e650a92de9c27'), 'name': 'Charlie', 'age': 26, 'gender': 'male'}
{'_id': ObjectId('656c2e02ab8e650a92de9c26'), 'name': 'Bob', 'age': 25, 'gender': 'male'}
{'_id': ObjectId('656c2e02ab8e650a92de9c25'), 'name': 'Alice', 'age': 24, 'gender': 'female'}
{'_id': ObjectId('656c2e02ab8e650a92de9c24'), 'name': 'Amanda', 'age': 18, 'gender': 'female'}


# 6.更新删除

In [15]:
# 更新1条数据
result = collection.update_one({'name': 'Amanda'}, {'$set': {'age': 19}})
result  # UpdateResult对象

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [16]:
result.matched_count, result.modified_count  # 匹配的记录数和修改的记录数

(1, 1)

In [17]:
for item in collection.find():
    print(item)  # age字段更新为19

{'_id': ObjectId('656c2e02ab8e650a92de9c24'), 'name': 'Amanda', 'age': 19, 'gender': 'female'}
{'_id': ObjectId('656c2e02ab8e650a92de9c25'), 'name': 'Alice', 'age': 24, 'gender': 'female'}
{'_id': ObjectId('656c2e02ab8e650a92de9c26'), 'name': 'Bob', 'age': 25, 'gender': 'male'}
{'_id': ObjectId('656c2e02ab8e650a92de9c27'), 'name': 'Charlie', 'age': 26, 'gender': 'male'}


In [18]:
# 年龄+3
collection.update_one({'name': 'Charlie'}, {'$inc': {'age': 3}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [19]:
# 查看更新后的结果
for item in collection.find():
    print(item)  # age字段+3

{'_id': ObjectId('656c2e02ab8e650a92de9c24'), 'name': 'Amanda', 'age': 19, 'gender': 'female'}
{'_id': ObjectId('656c2e02ab8e650a92de9c25'), 'name': 'Alice', 'age': 24, 'gender': 'female'}
{'_id': ObjectId('656c2e02ab8e650a92de9c26'), 'name': 'Bob', 'age': 25, 'gender': 'male'}
{'_id': ObjectId('656c2e02ab8e650a92de9c27'), 'name': 'Charlie', 'age': 29, 'gender': 'male'}


In [20]:
# 更新多条数据
collection.update_many({'age': {'$gt': 22}}, {'$inc': {'age': 1}})

UpdateResult({'n': 3, 'nModified': 3, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# 查看更新后的结果
for item in collection.find():
    print(item)  # age字段+1

{'_id': ObjectId('656c2e02ab8e650a92de9c24'), 'name': 'Amanda', 'age': 19, 'gender': 'female'}
{'_id': ObjectId('656c2e02ab8e650a92de9c25'), 'name': 'Alice', 'age': 25, 'gender': 'female'}
{'_id': ObjectId('656c2e02ab8e650a92de9c26'), 'name': 'Bob', 'age': 26, 'gender': 'male'}
{'_id': ObjectId('656c2e02ab8e650a92de9c27'), 'name': 'Charlie', 'age': 30, 'gender': 'male'}


In [22]:
# 删除1条数据
collection.delete_one({'name': 'Alice'})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [23]:
# 查看删除后的结果
for item in collection.find():
    print(item)

{'_id': ObjectId('656c2e02ab8e650a92de9c24'), 'name': 'Amanda', 'age': 19, 'gender': 'female'}
{'_id': ObjectId('656c2e02ab8e650a92de9c26'), 'name': 'Bob', 'age': 26, 'gender': 'male'}
{'_id': ObjectId('656c2e02ab8e650a92de9c27'), 'name': 'Charlie', 'age': 30, 'gender': 'male'}


In [24]:
# 删除多条数据
result = collection.delete_many({'age': {'$gt': 22}})
result

DeleteResult({'n': 2, 'ok': 1.0}, acknowledged=True)

In [25]:
result.deleted_count  # 删除的记录数

2

In [26]:
# 查看删除后的结果
for item in collection.find():
    print(item)

{'_id': ObjectId('656c2e02ab8e650a92de9c24'), 'name': 'Amanda', 'age': 19, 'gender': 'female'}
